### ПАКЕТ SURPRISE

используйте данные MovieLens 1M   

можно использовать любые модели из пакета   

получите RMSE на тестовом сете 0.87 и ниже   


In [14]:
import pandas as pd
import numpy as np

from surprise import KNNWithMeans, KNNBasic
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

Загрузим данные

In [2]:
movies = pd.read_csv('C:/Netology/Рекомендательные системы/movies.csv')
ratings = pd.read_csv('C:/Netology/Рекомендательные системы/ratings.csv')

In [27]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   title    58098 non-null  object
 2   genres   58098 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [25]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 847.0 MB


Объединим датафреймы movies и ratings  в один

In [3]:
movies_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_ratings.dropna(inplace=True)

In [36]:
movies_ratings[movies_ratings.userId == 8.0].title.unique()

array(['Grumpier Old Men (1995)', 'Father of the Bride Part II (1995)',
       'Sabrina (1995)', 'Dracula: Dead and Loving It (1995)',
       'Nixon (1995)', 'Mighty Aphrodite (1995)',
       "Mr. Holland's Opus (1995)", 'City Hall (1996)',
       'Down Periscope (1996)', 'Up Close and Personal (1996)',
       'Birdcage, The (1996)',
       'Star Wars: Episode IV - A New Hope (1977)', 'One Fine Day (1996)',
       'Sgt. Bilko (1996)', 'Mission: Impossible (1996)',
       'Spy Hard (1996)', 'Striptease (1996)', 'Jack (1996)',
       'Independence Day (a.k.a. ID4) (1996)', 'Kansas City (1996)',
       'Everyone Says I Love You (1996)', 'Associate, The (1996)',
       'Star Wars: Episode VI - Return of the Jedi (1983)',
       "I'm Not Rappaport (1996)", 'Star Trek: First Contact (1996)',
       "Preacher's Wife, The (1996)", 'My Fellow Americans (1996)',
       'Mars Attacks! (1996)', 'Jerry Maguire (1996)', 'Michael (1996)',
       'Fools Rush In (1997)'], dtype=object)

Создадим датасет для работы модуля surprise

In [4]:
dataset = pd.DataFrame({
    'uid': movies_ratings.userId,
    'iid': movies_ratings.title,
    'rating': movies_ratings.rating
})

In [39]:
dataset.head()

,uid,iid,rating
0,4.0,Toy Story (1995),4.0
1,10.0,Toy Story (1995),5.0
2,14.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),4.0
4,22.0,Toy Story (1995),4.0


In [41]:
ratings.rating.min()

0.5

In [42]:
ratings.rating.max()

5.0

In [5]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

Разделим данные на обучающую и тестовую выборки и обучим модель 

In [6]:
trainset, testset = train_test_split(data, test_size=.15)

In [7]:
algo = KNNBasic(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Проверим результаты предсказания

In [8]:
test_pred = algo.test(testset)

In [9]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9669


0.9668979047448928

In [13]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9720  0.9671  0.9731  0.9772  0.9733  0.9725  0.0032  
MAE (testset)     0.7502  0.7458  0.7498  0.7487  0.7492  0.7488  0.0016  
Fit time          1.08    1.12    1.10    1.09    1.09    1.10    0.02    
Test time         1.96    1.87    1.85    1.95    2.06  

{'test_rmse': array([0.97203575, 0.96707991, 0.97310719, 0.97717892, 0.97326287]),
 'test_mae': array([0.75022521, 0.74578346, 0.74983477, 0.74871705, 0.74923395]),
 'fit_time': (1.076845645904541,
  1.1227149963378906,
  1.0972044467926025,
  1.0918986797332764,
  1.0872526168823242),
 'test_time': (1.9639487266540527,
  1.8730437755584717,
  1.853562355041504,
  1.9464919567108154,
  2.0569236278533936)}

Сделаем тоже самое на модели SVD

In [15]:
algo2 = SVD()

In [17]:
trainset, testset = train_test_split(data, test_size=.25)

In [20]:
algo2.fit(trainset)
predictions = algo2.test(testset)

In [21]:
accuracy.rmse(predictions)

RMSE: 0.8719


0.8718919820353381

In [22]:
cross_validate(algo2, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8707  0.8728  0.8702  0.8766  0.8797  0.8740  0.0037  
MAE (testset)     0.6688  0.6728  0.6690  0.6733  0.6746  0.6717  0.0024  
Fit time          7.07    7.10    7.23    7.14    7.23    7.15    0.07    
Test time         0.23    0.31    0.31    0.26    0.23    0.27    0.03    


{'test_rmse': array([0.87066281, 0.87275238, 0.87017985, 0.87656095, 0.87972825]),
 'test_mae': array([0.66879975, 0.67284371, 0.66904286, 0.67332989, 0.67457351]),
 'fit_time': (7.073644638061523,
  7.095487594604492,
  7.232632637023926,
  7.137507915496826,
  7.2270355224609375),
 'test_time': (0.23264360427856445,
  0.3065683841705322,
  0.30674076080322266,
  0.2637019157409668,
  0.23496079444885254)}